In [313]:
import requests
import requests.auth
import json
from requests.auth import HTTPBasicAuth
import pandas as pd
from pandas.io.json import json_normalize

In [314]:
url = "https://api.crunchbase.com/api/v4/entities/organizations/crunchbase?user_key=INSERT KEY HERE"

user_key = {"user_key":"INSERT KEY HERE"}

response = requests.get(url,auth = HTTPBasicAuth("INSERT USER NAME HERE",user_key))

response

<Response [200]>

In [315]:
#Companies that raised between €5m and €100m

query = {
"field_ids": [
"identifier",
"categories",
"location_identifiers",
"short_description",
"founded_on",
"num_employees_enum",
"num_funding_rounds",
"equity_funding_total",
#"last_funding_at",
#"last_equity_funding_total",
#"last_equity_funding_type",
#"last_funding_type",
"funding_stage",
#"last_funding_at",
#"last_funding_total",
"num_founders",
"categories",
"website"
],
"limit": 1000,
"query": [
{
"type": "predicate",
"field_id": "funding_total",
"operator_id": "between",
"values": [
{"value":5000000,
"currency": "eur"},
    {"value": 100000000,
    "currency": "eur"}
]
},
{
"type": "predicate",
"field_id": "facet_ids",
"operator_id": "includes",
"values": [
"company"
]
}
]
}

In [316]:
def company_count(query):
    r = requests.post("https://api.crunchbase.com/api/v4/searches/organizations", params = user_key , json = query)
    result = json.loads(r.text)
    total_companies = result["count"]
    return total_companies
def url_extraction(query):
    global raw
    r = requests.post("https://api.crunchbase.com/api/v4/searches/organizations", params = user_key , json = query)
    result = json.loads(r.text)
    normalized_raw = json_normalize(result["entities"])
    raw = raw.append(normalized_raw,ignore_index=True)

In [295]:
requests.post("https://api.crunchbase.com/api/v4/searches/organizations", params = user_key , json = query)

<Response [200]>

In [317]:
raw=pd.DataFrame()
#comp_count = company_count(query)
data_acq = 0
# data_acq
while data_acq < 10:
    if data_acq != 0:
        last_uuid = raw.uuid[len(raw.uuid)-1]
        query["after_id"] = last_uuid
        url_extraction(query)
        data_acq = len(raw.uuid)
    else:
        if "after_id" in query:
            query = query.pop("after_id")
            url_extraction(query)
            data_acq = len(raw.uuid)
        else:
            url_extraction(query)
            data_acq = len(raw.uuid)

<ipython-input-316-b21038c9d6ac>:10: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  normalized_raw = json_normalize(result["entities"])


In [318]:
raw.head()

,uuid,properties.founded_on.precision,properties.founded_on.value,properties.website.value,properties.equity_funding_total.value_usd,properties.equity_funding_total.currency,properties.equity_funding_total.value,properties.identifier.permalink,properties.identifier.image_id,properties.identifier.uuid,properties.identifier.entity_def_id,properties.identifier.value,properties.short_description,properties.num_employees_enum,properties.num_funding_rounds,properties.categories,properties.location_identifiers,properties.num_founders,properties.funding_stage
0,01a2a666-577c-1673-eba6-74c02aaf2e80,day,2010-08-01,https://www.crowdcube.com/,44568167.0,GBP,30686010.0,crowdcube,smdxzio4kigyx52cnlpu,01a2a666-577c-1673-eba6-74c02aaf2e80,organization,Crowdcube,Crowdcube is an equity crowdfunding platform f...,c_00051_00100,12,"[{'entity_def_id': 'category', 'permalink': 'b...","[{'permalink': 'exeter-devon', 'uuid': 'c5f8ba...",4.0,NaN
1,2c1f42ac-94ed-c6ad-8881-cfacb63ccd08,year,2016-01-01,http://www.encantosbrands.com,8500000.0,USD,8500000.0,encantos,hs4n3yh78prxmntafs2x,2c1f42ac-94ed-c6ad-8881-cfacb63ccd08,organization,Encantos,Encantos is an ed-tech company creating direct...,c_00051_00100,3,"[{'entity_def_id': 'category', 'permalink': 'a...","[{'permalink': 'culver-city-california', 'uuid...",4.0,seed
2,3dfd8d9c-7480-79e8-a983-dcf1d0f8e1a5,year,2007-01-01,https://www.twitch.tv/,35000000.0,USD,35000000.0,twitch,v1475259321/bv2qlltonsln3ndrk8ws.png,3dfd8d9c-7480-79e8-a983-dcf1d0f8e1a5,organization,Twitch,Twitch is social video platform for gamers whe...,c_00251_00500,3,"[{'entity_def_id': 'category', 'permalink': 's...","[{'permalink': 'san-francisco-california', 'uu...",6.0,m_and_a
3,8092160c-26ed-22fc-668a-9bdbb8af5c32,year,2007-01-01,https://aclima.io,64000000.0,USD,64000000.0,aclima,qjxuxh0su7kjw3agxozj,8092160c-26ed-22fc-668a-9bdbb8af5c32,organization,Aclima,Aclima delivers hyperlocal air quality data an...,c_00051_00100,2,"[{'entity_def_id': 'category', 'permalink': 'a...","[{'permalink': 'san-francisco-california', 'uu...",2.0,early_stage_venture
4,813b4633-dc4a-4985-86bd-1bcc4ce49378,year,2018-01-01,https://careerkarma.com/,11937500.0,USD,11937500.0,career-karma,szarnyh6a3q6owbb8ohw,813b4633-dc4a-4985-86bd-1bcc4ce49378,organization,Career Karma,Career Karma helps people find career advice a...,c_00051_00100,3,"[{'entity_def_id': 'category', 'permalink': 'c...","[{'permalink': 'san-francisco-california', 'uu...",3.0,early_stage_venture


In [319]:
from operator import itemgetter
employee_range = {
"c_00001_00010": "1-10",
"c_00011_00050": "11-50",
"c_00051_00100": "51-100",
"c_00101_00250": "101-250",
"c_00251_00500": "251-500",
"c_00501_01000": "501-1000",
"c_01001_05000": "1001-5000",
"c_05001_10000": "5001-10000",
"c_10001_max": "10001+"}

master = pd.DataFrame()
master["uuid"] = raw["uuid"]
master["company"] = raw["properties.identifier.value"]
master["description"] = raw["properties.short_description"]
master["categories"] = raw["properties.categories"].apply(lambda x: list(map(itemgetter('value'), x)if isinstance(x, list) else ["Not found"])).apply(lambda x : ",".join(map(str, x)))
master["location"] = raw["properties.location_identifiers"].apply(lambda x: list(map(itemgetter('value'), x)if isinstance(x, list) else ["Not found"])).apply(lambda x : ",".join(map(str, x)))
new = master["location"].str.split(",", n = 4, expand = True)
master["country"]= new[2]
master["continent"]= new[3]
master["num_of_employees"] = raw["properties.num_employees_enum"].map(employee_range)
master["website"] = raw["properties.website.value"]
master["Founded"] = raw["properties.founded_on.value"] #you could also just get the year
master["Funding Rounds"] = raw["properties.num_funding_rounds"]
master["Total Funding in USD"] = raw["properties.equity_funding_total.value_usd"]
master["Funding Range"] = pd.cut(raw["properties.equity_funding_total.value_usd"], bins=[0,5000000,15000000,25000000,35000000,45000000,55000000,65000000,75000000,85000000,100000000], labels=["<5m","5-15m","15-25m","25-35m","35-45m","45-55m","55-65m","65-75m","75-85m","85-100m"])
master["Funding Stage"] = raw["properties.funding_stage"]
master["Number Founders"] = raw["properties.num_founders"]
#master=master.fillna("NA")

In [299]:
master.head()

,uuid,company,description,categories,location,country,continent,num_of_employees,website,Founded,Funding Rounds,Total Funding in USD,Funding Range,Funding Stage,Number Founders
0,01a2a666-577c-1673-eba6-74c02aaf2e80,Crowdcube,Crowdcube is an equity crowdfunding platform f...,"Business Development,Finance,Financial Service...","Exeter,Devon,United Kingdom,Europe",United Kingdom,Europe,51-100,https://www.crowdcube.com/,2010-08-01,12,44568167.0,35-45m,NaN,4.0
1,2c1f42ac-94ed-c6ad-8881-cfacb63ccd08,Encantos,Encantos is an ed-tech company creating direct...,"Apps,Children,Consumer,Digital Entertainment,E...","Culver City,California,United States,North Ame...",United States,North America,51-100,http://www.encantosbrands.com,2016-01-01,3,8500000.0,5-15m,seed,4.0
2,3dfd8d9c-7480-79e8-a983-dcf1d0f8e1a5,Twitch,Twitch is social video platform for gamers whe...,"Social Media,Video,Video Games,Video Streaming","San Francisco,California,United States,North A...",United States,North America,251-500,https://www.twitch.tv/,2007-01-01,3,35000000.0,25-35m,m_and_a,6.0
3,8092160c-26ed-22fc-668a-9bdbb8af5c32,Aclima,Aclima delivers hyperlocal air quality data an...,"Analytics,Enterprise Software,Location Based S...","San Francisco,California,United States,North A...",United States,North America,51-100,https://aclima.io,2007-01-01,2,64000000.0,55-65m,early_stage_venture,2.0
4,813b4633-dc4a-4985-86bd-1bcc4ce49378,Career Karma,Career Karma helps people find career advice a...,"Career Planning,Social Network,Training","San Francisco,California,United States,North A...",United States,North America,51-100,https://careerkarma.com/,2018-01-01,3,11937500.0,5-15m,early_stage_venture,3.0


In [301]:
master.to_csv("data.csv")